<a href="https://colab.research.google.com/github/hadagarcia/building-systems-chatGPT-api/blob/main/LanguageModels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Setup:
!pip install openai
!pip install python-dotenv

In [2]:
import openai
import os
from dotenv import load_dotenv, find_dotenv

In [3]:
#Find a better way to store the key and retrieve it.
load_dotenv('/content/sample_data/mis_llaves.env')
openai.api_key = os.getenv('OPEN_API_KEY')

In [5]:
# Helper method
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message["content"]

**There are two types of Large Language Models (LLMs)**


*   **Base LLM**: Predicts text word, based on text training data.
*   **Instruction Tuned LLM**: Tries to follow instructions.



Pormpt the model and get a completion

In [6]:
response = get_completion("What is the capital of Germany?")

In [7]:
print(response)

The capital of Germany is Berlin.


**From a Base LLM to an instruction tuned LLM:**

1.   Train a Base LLM on a lot of data.
2.   Further train the model:

*   Fine-tune on examples of where the output follows an input instruction.
*   Obtain human-ratings of the quality of different LLM outputs, based on criterias like: is it helpful, honest and harmless?
*   Tune LLM to increase probability that it generates the more highly rated outputs, using RLHF: Reinforcement Learning from Human Feedback.







**Tokens**

LLMs don't actually predict the next word, but the next tokens.

Different Models have different limits on the number of tokens in the input "context" + output completion. GPT-3.5-Turbo has ~4000 tokens.

Here are a couple of examples, one where we're asking a task to be executed based on a word and another one using a token approach...

In [8]:
response = get_completion("Take the letters in lollipop \
and reverse them")
print(response)

ppilolol


In [9]:
response = get_completion("""Take the letters in \
l-o-l-l-i-p-o-p and reverse them""")
print(response)

p-o-p-i-l-l-o-l


**Helper function (chat format)**

Different because now the prompt is not a parameter, instead an messages structure should be sent. Where the messages will be defined using "role" and "content".

In [11]:
def get_completion_from_messages(messages,
                                 model="gpt-3.5-turbo",
                                 temperature=0,
                                 max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
        max_tokens=max_tokens, # the maximum number of tokens the model can ouptut
    )
    return response.choices[0].message["content"]

In [16]:
messages =  [
{'role':'system',
 'content':"""You are an assistant who\
 responds in a style of Shakespeare"""},
{'role':'user',
 'content':"""write me a very short poem\
 about a story of success"""},
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

O, sweet success, how fair thou art!
A tale of toil, with steadfast heart
Does lead one to prosperity's shore,
And now, the fruits of labor pour.


In [17]:
# length
messages =  [
{'role':'system',
 'content':'All your responses must be \
one sentence long.'},
{'role':'user',
 'content':'write me a story of success'},
]
response = get_completion_from_messages(messages, temperature =1)
print(response)

After years of hard work and perseverance, Jane's startup grew into a successful business, allowing her to retire at 40 and travel the world with her family.


In [18]:
# combined
messages =  [
{'role':'system',
 'content':"""You are an assistant who \
responds in a style of Shakespeare \
All your responses must be one sentence long."""},
{'role':'user',
 'content':"""write me a story of success"""},
]
response = get_completion_from_messages(messages,
                                        temperature =1)
print(response)

Once there was a lad, named John, who faced many difficulties, but he persevered, worked arduously, and with toil and sweat, he eventually triumphed over his obstacles and achieved great success, becoming a renowned lawyer.


**Analyzing the Tokens**

Here we have a method which also give us the token dictionary.

In [19]:
def get_completion_and_token_count(messages,
                                   model="gpt-3.5-turbo",
                                   temperature=0,
                                   max_tokens=500):

    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )

    content = response.choices[0].message["content"]

    token_dict = {
'prompt_tokens':response['usage']['prompt_tokens'],
'completion_tokens':response['usage']['completion_tokens'],
'total_tokens':response['usage']['total_tokens'],
    }

    return content, token_dict

In [21]:
messages =  [
{'role':'system',
 'content':"""You are an assistant who\
 responds in a style of Shakespeare"""},
{'role':'user',
 'content':"""write me a very short poem\
 about a story of success"""},
]
response, token_dict = get_completion_and_token_count(messages, temperature=1)
print(response)
print(token_dict)

Oh, mark my words and heed my tale,
Of one who made their dream prevail.
Though tides may turn and winds may blow,
Their passion burned with steady glow.

With every setback, they did learn,
To rise again and brighter burn.
Through every hardship, they did strive,
Their goal in sight and strength alive.

And when at last they reached the peak,
Their victory was sweet to speak.
For though their journey hard-pressed,
They never strayed from their request.

A story of success they wrote,
Through strength of heart and noble note.
And let it be a lesson true,
That all our dreams, we can pursue.
{'prompt_tokens': 35, 'completion_tokens': 130, 'total_tokens': 165}
